In [1]:
# General imports
import os
import sys
import numpy as np

# Gymnasium
import gymnasium as gym

# Sinergym
import sinergym
from sinergym.utils.rewards import ExpReward
from src.sinergym_wrapper import SinergymWrapper

In [2]:
# Add the EnergyPlus path to the system path
sys.path.append('./EnergyPlus-23-1-0')
# Set the EnergyPlus path as an environment variable
os.environ['EPLUS_PATH'] = './EnergyPlus-23-1-0'

In [3]:
# Environment ID
id = "Eplus-5zone-hot-continuous-stochastic-v1"

# Weather
weather_files = ['USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw']

# mu, sigma and theta for the weather variability
# In the original version, weather_variability was a triple (mu, sigma, theta)
# that only affected the drybulb (outdoor temperature), while now it is a dictionary
# with the mean, sigma and theta for each weather variable we want to vary
# NOTE: Check apply_weather_variability method in CustomModelJSON class
weather_variability = {
    'drybulb': np.array([5.53173187e+00, 0.00000000e+00, 2.55034944e-03]), 
    'relhum': np.array([1.73128872e+01, 0.00000000e+00, 2.31712760e-03]), 
    'winddir': np.array([7.39984654e+01, 0.00000000e+00, 4.02298013e-04]), 
    'dirnorrad': np.array([3.39506556e+02, 0.00000000e+00, 9.78192172e-04]), 
    'windspd': np.array([1.64655725e+00, 0.00000000e+00, 3.45045547e-04])}

# Reward function and kwargs
reward = ExpReward
reward_kwargs = {
    'temperature_variables': ['air_temperature'],
    'energy_variables': ['HVAC_electricity_demand_rate'],
    'range_comfort_winter': [20, 23],
    'range_comfort_summer': [23, 26],
    'energy_weight': 0.1
}

In [4]:
# Create the environment
env = gym.make(
    id=id,
    weather_files=weather_files,
    reward=reward,
    reward_kwargs=reward_kwargs,
    weather_variability=weather_variability
)

# Wrap the environment (to account for the different weather variability)
env =  SinergymWrapper(env)  

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-hot-continuous-stochastic-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/home/luigi/Documents/SmartEnergyOptimizationRL/Eplus-env-5zone-hot-continuous-stochastic-v1-res1]
[MODELING] (INFO) : Model Config is correct.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter available names
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 0, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode:

In [5]:
env.unwrapped.info()


    #==================================================================================#
                                ENVIRONMENT NAME: 5zone-hot-continuous-stochastic-v1
    #==================================================================================#
    #----------------------------------------------------------------------------------#
                                ENVIRONMENT INFO:
    #----------------------------------------------------------------------------------#
    - Building file: /home/luigi/Documents/SmartEnergyOptimizationRL/.venv/lib/python3.10/site-packages/sinergym/data/buildings/5ZoneAutoDXVAV.epJSON
    - Zone names: ['PLENUM-1', 'SPACE1-1', 'SPACE2-1', 'SPACE3-1', 'SPACE4-1', 'SPACE5-1']
    - Weather file(s): ['USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw']
    - Current weather used: /home/luigi/Documents/SmartEnergyOptimizationRL/.venv/lib/python3.10/site-packages/sinergym/data/weather/USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw
    - Episodes executed: 

In [6]:
for i in range(1):
    obs, info = env.reset()
    rewards = []
    terminated = False
    current_month = 0
    while not terminated:
        a = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(a)
        if info['month'] != current_month:  # display results every month
            current_month = info['month']
            print("\n\nOBSERVATION: ", obs)
            print("REWARD: ", reward)
            print("TERMINATED: ", terminated)
            print("TRUNCATED: ", truncated)
            print("INFO: ", info)
        break

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [5zone-hot-continuous-stochastic-v1] [Episode 1]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/home/luigi/Documents/SmartEnergyOptimizationRL/Eplus-env-eplus-env-v1-res1/Eplus-env-sub_run1]
[MODELING] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODELING] (INFO) : Adapting weather to building model. [USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw]
[ENVIRONMENT] (INFO) : Saving episode output path... [/home/luigi/Documents/SmartEnergyOptimizationRL/Eplus-env-eplus-env-v1-res1/Eplus-env-sub_run1/output]
[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/home/luigi/Documents/SmartEnergyOptimizationRL/Eplus-env-eplus-env-v1-res1/Eplus-env-sub_run1/USA_AZ_Davis-Monthan.AFB.722745_TMY3_Randomized.epw', '-d', '/home/luigi/Do

In [7]:
from stable_baselines3 import PPO

# Create the PPO agent
model = PPO('MlpPolicy', env, verbose=1)

# Train the agent
model.learn(total_timesteps=10000)

# Save the agent
model.save("ppo_agent")

# To load the trained agent
# model = PPO.load("ppo_agent")


2024-02-06 18:24:55.437214: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-06 18:24:55.467703: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-06 18:24:55.467729: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-06 18:24:55.468654: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-06 18:24:55.474625: I tensorflow/core/platform/cpu_feature_guar

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [5zone-hot-continuous-stochastic-v1] [Episode 2]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/home/luigi/Documents/SmartEnergyOptimizationRL/Eplus-env-eplus-env-v1-res1/Eplus-env-sub_run2]
[MODELING] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODELING] (INFO) : Adapting weather to building model. [USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw]
[ENVIRONMENT] (INFO) : Saving episode output path... [/home/luigi/Documents/SmartEnergyOptimizationRL/Eplus-env-eplus-env-v1-res1/Eplus-env-sub_run2/output]
[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/home/luigi/Documents/SmartEnergyOptimizationRL/Eplus-env-eplus-env-v1-res1

Progress: |***************************************************************************************************| 99%


In [8]:
env.close()

Progress: |***************************************************************************************************| 99%


[ENVIRONMENT] (INFO) : Environment closed. [5zone-hot-continuous-stochastic-v1]
